# Home Assignment 1 - Crawler

**Author:**    Danis Alukaev <br/>
**Email:**      d.alukaev@innopolis.university <br/>
**Group:**      B19-DS-01

## 1.0. Related example

This code shows `wget`-like tool written in python. Run it from console (`python wget.py`), make it work. Check the code, reuse, and modify for your needs.

In [ ]:
import argparse
import os
import re
import requests


def wget(url, filename):
    # allow redirects - in case file is relocated
    resp = requests.get(url, allow_redirects=True)
    # this can also be 2xx, but for simplicity now we stick to 200
    # you can also check for `resp.ok`
    if resp.status_code != 200:
        print(resp.status_code, resp.reason, 'for', url)
        return
    
    # just to be cool and print something
    print(*[f"{key}: {value}" for key, value in resp.headers.items()], sep='\n')
    print()
    
    # try to extract filename from url
    if filename is None:
        # start with http*, ends if ? or # appears (or none of)
        m = re.search("^http.*/([^/\?#]*)[\?#]?", url)
        filename = m.group(1)
        if not filename:
            raise NameError(f"Filename neither given, nor found for {url}")

    # what will you do in case 2 websites store file with the same name?
    if os.path.exists(filename):
        raise OSError(f"File {filename} already exists")
    
    with open(filename, 'wb') as f:
        f.write(resp.content)
        print(f"File saved as {filename}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='download file.')
    parser.add_argument("-O", type=str, default=None, dest='filename', help="output file name. Default -- taken from resource")
    parser.add_argument("url", type=str, default=None, help="Provide URL here")
    args = parser.parse_args()
    wget(args.url, args.filename)

### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [15] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [1]:
import requests
from urllib.parse import quote
from os.path import exists, join 

class Document:
    
    def __init__(self, url):
        """
        TODO: check for the mp3 file.
        """
        self.url = url
        self.content = None
        self.dir = "./data"
        
    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
    
    def download(self):
        """
        Downloads binary data and stores it in the content attribute.

        :return: boolean whether the data was downloaded successfully.
        """
        response = requests.get(self.url, allow_redirects=True)
        if response.status_code == 200:
            self.content = response.content
            return True
        print(f"Download failed for {self.url}: {response.status_code} ({response.reason})")
        return False
    
    def _get_filepath(self):
        """
        Returns path to the file storing content of url.

        :return: filepath
        """
        filename = quote(self.url, '')
        filepath = join(self.dir, filename)
        return filepath


    def persist(self):
        """
        Saves content in the local directory data.
        Primarily, it is used for caching.

        :return: boolean whether the data was stored successfully.

        TODO: (?) check if the version exists or should be updated (new file for new content)
        """
        try:
            filepath = self._get_filepath()
            with open(filepath, 'wb') as f:
                f.write(self.content)
        except Exception as e:
            print(e)
            return False
        return True
            
    def load(self):
        """
        Loads data from the local directory.

        :return: boolean whether the data was loades successfully.
        """
        filepath = self._get_filepath()
        if exists(filepath):
            with open(filepath, 'r') as f:
                self.content = f.read()
            return True
        return False

### 1.1.1. Tests ###

In [2]:
doc = Document('http://sprotasov.ru/data/iu.txt')

doc.get()
assert doc.content, "Document download failed"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

## 1.3. [10] Parse HTML ##
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
- `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
- `self.images` list of images met in a document. Again, links can be relative to current page.
- `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [3]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse


class HtmlDocument(Document):
    
    def parse(self):
        self.soup = BeautifulSoup(self.content, 'html.parser')

        self.anchors = self._get_anchors()
        self.images = self._get_images()
        self.text = self._get_text()

    def _get_anchors(self):
        """
        Creates a list of anchor tuples ('text', 'url') met in a document.
        
        :return: list of anchor tuples.
        """
        absolute_link_marker = "//"
        anchors = self.soup.find_all('a')
        result = []
        for anchor in anchors:
            link = anchor['href']
            if absolute_link_marker not in anchor:
                link = urllib.parse.urljoin(self.url, anchor['href'])
            entry = (anchor.text, link)
            result.append(entry)
        return result
        
    
    def _get_images(self):
        """
        Creates a list of links to images met in document.

        :return: list of links to images.

        TODO: (?) should links be unique
        """
        absolute_link_marker = "//"
        images = self.soup.find_all("img")
        result = []
        for entry in images:
            result.append(entry)
            if absolute_link_marker not in entry:
                result[-1] = urllib.parse.urljoin(self.url, entry["src"])
        return result
    
    def _get_text(self):
        """
        Converts document content to plain text.
        
        :return: string with plain text.
        """
        texts = self.soup.findAll(text=True)
        containers = ['head', 'title', 'style', 'script', 'meta', '[document]']

        cleaned_texts = []
        for element in texts:
            parent = element.parent.name 
            if parent in containers or isinstance(element, Comment):
                continue
            cleaned_texts.append(element.strip())
        plain = " ".join(cleaned_texts).strip()
        return plain

### 1.3.1. Tests ###

In [4]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()

assert "just few links" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/gb.svg" in doc.images, "Error parsing images"
assert any(p[1] == "https://twitter.com/07C3" for p in doc.anchors), "Error parsing links"

## 1.4. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose). 

**Criteria of success**: 
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained inside `<body>` tag only.

In [ ]:
!pip install nltk

In [5]:
import re
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize


class HtmlDocumentTextData:
    
    def __init__(self, url):
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()
        
    
    def get_sentences(self):
        """
        Creates a list of sentences from the <body> tag of the content.
        :return: list of sentences.

        TODO: (?) what is the sentence
        """
        body_strings = doc.doc.soup.body.strings
        body = re.sub('\s{2,}', ' ', " ".join(list(body_strings)).replace('\n', ' ')).strip()   
        result = sent_tokenize(body)
        return result
    
    def get_word_stats(self):
        """
        Creates a counter of each word in the content text.
        :return: counter.
        """
        counter = Counter()
        words = [word.lower() for word in word_tokenize(self.doc.text) if word not in [',', '.']]
        for word in words:
            counter[word] += 1
        return counter

### 1.4.1. Tests ###

In [6]:
doc = HtmlDocumentTextData("https://innopolis.university/")
print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис should be among most common'

[('и', 60), ('в', 35), ('иннополис', 21), ('по', 16), ('университет', 12), ('на', 11), ('области', 10), ('лаборатория', 10), ('университета', 9), ('центр', 9)]


## 1.5. [15] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [ ]:
from queue import Queue

class Crawler:
    
    def crawl_generator(self, source, depth=1):
        #TODO return real crawling results. Don't forget to process failures
        for i in range(3):
            yield HtmlDocumentTextData(source)

### 1.5. Tests ###

In [ ]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'